<a href="https://colab.research.google.com/github/DiyRex/Torrent-to-G-Drive-/blob/main/Torrent_To_G_Drive_File_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##<h1>&nbsp;<center><img style="display: block; margin-left: auto; margin-right: auto;" src="https://github.com/DiyRex/Torrent-to-G-Drive-/blob/main/torrentdrive.png?raw=true" alt="" width="90" height="100" /></center>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; <strong>Torrent To G-Drive File Downloader</strong></h1><p><strong>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; - Script by DiyRex-</strong></p>





--------------------

----------------------









In [4]:
#@markdown <br><center><img src='https://github.com/DiyRex/Torrent-to-G-Drive-/blob/main/google_drive_new_logo-512.png?raw=true' height="100" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT G-Drive" #@param ["MOUNT G-Drive", "UNMOUNT G-Drive"]
#Mount your G-drive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT G-Drive":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT G-Drive":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive




---



---









In [2]:
#@markdown <ul>
#@markdown <li>
#@markdown <h2><em> Install lbtorrent and Initialize Session</em></h2>
#@markdown &nbsp;&nbsp;</li>
#@markdown </ul>
### Install lbtorrent and Initialize Session

!apt install python3-libtorrent
!python -m pip install lbry-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []


Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-libtorrent is already the newest version (1.1.5-1build1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.



---

---









<ul>
<li>
<h2><em> You Can Choose Following Options&nbsp;</em></h2>
&nbsp;&nbsp;</li>
</ul>

1.   Add From Torrent File
2.   Add From Magnet Link





---

---









In [ ]:
#@markdown <ul>
#@markdown <li>
#@markdown <h2 class="text-cell-section-header"><em>Add From Torrent File</em></h2>
#@markdown <h2>&nbsp;</h2>
#@markdown </li>
#@markdown </ul>
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))



---

---





In [3]:
#@markdown <ul>
#@markdown <li>
#@markdown <h2>Add From Magnet Link</h2>
#@markdown </li>
#@markdown </ul>
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:bd2483a16bd344dd51d6342d188f9c93328f6051&dn=rutor.info_Far+Cry+3%3A+Deluxe+Edition+%2B+Blood+Dragon+%5Bv+1.05%2F1.02%5D+%282012-2013%29+PC+%7C+RePack+%D0%BE%D1%82+FitGirl&tr=udp://opentor.org:2710&tr=udp://opentor.org:2710&tr=http://retracker.local/announce
Enter Magnet Link Or Type Exit: exit




---

---





In [4]:
#@markdown <ul style="list-style-type: square;">
#@markdown <li>
#@markdown <h1><strong>Start Download</strong></h1>
#@markdown </li>
#@markdown </ul>
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

Far Cry 3 - Duology [FitGirl Repack] complete
